In [ ]:
import csv, json, re
from pandas.io.json._table_schema import build_table_schema

In [ ]:
def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []
      
    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf) 

        #convert each csv row into python dict
        for row in csvReader: 
            #add this python dict to json array
            if len(row['ticker']) < 4:
                jsonArray.append(row['ticker'])
    
    jsonArray = re.sub(' ', '', str(jsonArray)) 
    jsonArray = re.sub('\'', '', str(jsonArray))
    jsonArray = jsonArray[1:-1]
    #convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as f: 
        f.write(jsonArray)

In [ ]:
csvFilePath = r'/home/jazzdung/Library/Downloads/vn_stock_listing_companies_2023-05-20.csv'
jsonFilePath = r'symbol.txt'
org = csv_to_json(csvFilePath, jsonFilePath)

In [ ]:
def write_configured_catalog(df_list, file):
    final_json = """
    {
    "streams": [
    """

    for df in df_list:
        schema = build_table_schema(df)
        schema = schema['fields']
        schema = str(schema[1:])
        schema = re.sub("\[", "", schema)
        schema = re.sub("\]", "", schema)
        schema = re.sub("'", "\"", schema)
        schema = re.sub("{\"name\": ", "", schema)
        schema = re.sub(", \"type\"", ": {\"type\"", schema)


        header = """
        {
            "stream": {
            "name": "balance_sheet",
            "description": "Stock data price history",
            "json_schema": {
                "$schema": "http://json-schema.org/draft-04/schema#",
        """

        ending = """
            },
            "supported_sync_modes": ["full_refresh"]
            },
            "sync_mode": "full_refresh",
            "destination_sync_mode": "append"
        }
        """

        schema = header + schema + ending

        final_json += schema
    
    final_json += "] }"
    f = open(file, "w")
    f.write(final_json)
    f.close()

    return schema

In [ ]:
balancesheet = financial_flow('TCB', 'balancesheet','yearly')
incomestatement = financial_flow('TCB', 'incomestatement','yearly')
cashflow = financial_flow('TCB', 'cashflow','yearly')

write_configured_catalog([generalrating], "integration_tests/configured_catalog.json")